In [1]:
import traceback
import numpy as np
import pandas as pd
#import datatable as dt
import gc
import time
import os
import talib as ta
from script.crypto_API import *
import IPython

import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp

In [14]:
ASSET_DETAILS_CSV = './data/asset_details.csv'
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
# sup_train=pd.read_csv('./data/supplemental_train.csv')
# assets_order = sup_train.Asset_ID[:14]
# assets_order = dict((t,i) for i,t in enumerate(assets_order))
mod_order = [3, 2, 0, 1, 4, 5, 7, 6, 8, 9, 10, 13, 12, 11]
features_list = ['Count', 'Open', 'High', 'Low', 'Close', 'Volume',
        'lr_15', 'Mkt_lrt_15', 'Crypto_Index', 'beta_6h',
       'lr_15_resid_6h', 'beta_2d', 'lr_15_resid_2d', 'lrtn_index_50', 'slowK',
       'slowD', 'fastK', 'fastD', 'rsi_30', 'macd_10_60', 'macd_signal_15',
       'macd_hist', 'adx_50', 'vol_sum_15', 'std_lr_15_30',
       'std_Mkt_lrt_15_10', 'std_Crypto_Index_30']

psets={'std_lr_15': 30, 'std_Mkt_lrt_15': 10, 'std_Crypto_Index': 30, 
            'rsi': 30, 'adx': 50, 'macd_sig': 15, 'macd_s': 10, 'macd_l': 60, 'lrtn': 50, 
            'fastk2': 10, 'fastk1': 15, 'beta_s': '6h', 'beta_l': '2d', 'vol_sum': 15}


version = 1
retrained = False #set to False to get correct out-train score
pre_minute=101 #psets['adx']*2+1 #for smoothing sensitive indicators
lstm_window = 15

MOD_FOLDER = "./trainedNN" + f"./kaggleLSTM_{version}"
MOD_FOLDER

'./kaggleLSTM_1'

In [15]:
def masked_cosine(y_true, y_pred):
    mask = tf.math.not_equal(y_true, 0.)
    y_true_masked = tf.boolean_mask(y_true, mask)
    y_pred_masked = tf.boolean_mask(y_pred, mask)
    return tf.keras.losses.cosine_similarity(y_true_masked, y_pred_masked)

def nonmasked_cosine(y_true, y_pred):
    '''
    2.x uses the negative similarity because it's used as a loss function
    '''
    return tf.keras.losses.cosine_similarity(y_true, y_pred)

def Correlation(y_true,y_pred): 
    return tf.math.abs(tfp.stats.correlation(y_pred,y_true, sample_axis=None, event_axis=None))

In [16]:
from os.path import exists
if exists(MOD_FOLDER):
    if version<1:
        model = keras.models.load_model(MOD_FOLDER, 
                                    custom_objects={"masked_cosine": masked_cosine, 'Correlation':Correlation}, 
                                    compile =False)
    else:
        model = keras.models.load_model(MOD_FOLDER, 
                                    custom_objects={"nonmasked_cosine": nonmasked_cosine, 'Correlation':Correlation}, 
                                    compile =False)        
    IPython.display.clear_output()
    print(f'finish loading the LSTM from {MOD_FOLDER}')
else:
    raise ValueError

finish loading the LSTM from ./kaggleLSTM_1


In [17]:

def log_return(series, periods=5):
    return np.log(series).diff(periods)

def lag_features(df, fastk1,fastk2,adx,macd_s,macd_l,macd_sig,vol_sum,rsi,std_Crypto_Index,std_lr_15,std_Mkt_lrt_15, **kwargs):    
    if vol_sum >0 :
        df[f'vol_sum_{vol_sum}'] = ta.SMA(df['Volume'],vol_sum)*vol_sum
    ####TECH indicators
    df['slowK'], df['slowD'] = ta.STOCH(df.High, df.Low, df.Close, 
                                        fastk_period=fastk1, slowk_period=int(3*fastk1/5), slowd_period=int(3*fastk1/5),
                                        slowk_matype=0, slowd_matype=0)
    df['fastK'], df['fastD'] = ta.STOCHF(df.High, df.Low, df.Close,
                                         fastk_period=fastk2, fastd_period=int(3*fastk2/5), 
                                         fastd_matype=0)
    df[f'macd_{macd_s}_{macd_l}'],df[f'macd_signal_{macd_sig}'], df['macd_hist'] = \
                ta.MACD(df['Close'],fastperiod=macd_s, slowperiod=macd_l, signalperiod=macd_sig)
    ##smoothing sensitive ADX 150, RSI250, AD unKNOWN
    df[f'rsi_{rsi}'] = ta.RSI(df['Close'], timeperiod=rsi)
    df[f'adx_{adx}'] = ta.ADX(df['High'], df['Low'],df['Close'], timeperiod=adx)

    ####std volatility
    df[f'std_lr_15_{std_lr_15}'] = ta.STDDEV(df.lr_15,timeperiod=std_lr_15, nbdev=1)
    df[f'std_Mkt_lrt_15_{std_Mkt_lrt_15}'] = ta.STDDEV(df.Mkt_lrt_15,timeperiod=std_Mkt_lrt_15, nbdev=1)
    df[f'std_Crypto_Index_{std_Crypto_Index}'] = ta.STDDEV(df.Crypto_Index,timeperiod=std_Crypto_Index, nbdev=1)

def beta_resid(df, width): 
    b = ((ta.MULT(df.Mkt_lrt_15,df.lr_15).mean())/ \
        (ta.MULT(df.Mkt_lrt_15,df.Mkt_lrt_15).mean()))
    if b in [np.nan,np.inf,-np.inf]:
        b=0
    return b 
def get_features(df_feat, psets):
    pd.options.mode.chained_assignment = None  # default='warn'
    df_feat[f"lr_15_resid_{psets['beta_s']}"] = ta.SUB(df_feat.lr_15, ta.MULT(df_feat[f"beta_{psets['beta_s']}"], df_feat.Mkt_lrt_15)).rename(f"lr_15_resid_{psets['beta_s']}")
    df_feat[f"lr_15_resid_{psets['beta_l']}"] = ta.SUB(df_feat.lr_15, ta.MULT(df_feat[f"beta_{psets['beta_l']}"], df_feat.Mkt_lrt_15)).rename(f"lr_15_resid_{psets['beta_l']}")
    df_feat[f"lrtn_index_{psets['lrtn']}"] = log_return(df_feat.Crypto_Index, psets['lrtn'])
    lag_features(df_feat, **psets)
    return df_feat

## API submission

In [18]:
start = datestring_to_timestamp('2021-06-13T00:00:00')#1623542400
#end = datestring_to_timestamp('2021-09-21T00:00:00')#1632182400
end = datestring_to_timestamp('2021-06-13T01:00:00') #1623628800
train_df = read_csv_slice(file_path='./data/train.csv', 
                          use_window=[start, end])


In [19]:
def beta_window(beta):
    num, unit = int(beta[:-1]),beta[-1]
    if unit == 'h':
        width = 60*num
    elif unit == 'd':
        width = 60*24*num
    return width

beta_sw = beta_window(psets['beta_s'])
beta_lw = beta_window(psets['beta_l'])
beta_lw

2880

In [20]:
pre_minute_beta =  beta_lw + 15#for beta calculations
pre_minute, pre_minute_beta

(101, 2895)

In [21]:
add_weight_map = dict(zip(df_asset_details.Asset_ID, 
                        df_asset_details.Weight/df_asset_details.Weight.sum()))

###load sup_train
sup_train = pd.read_csv('./data/supplemental_train.csv')
sup_train = sup_train.sort_values('timestamp').set_index("timestamp")
ind = sup_train.index.unique()
###consistent timestamp for all 14 assets
def reindex(df):
    df = df.reindex(range(ind[0],ind[-1]+60,60),method='nearest')
    df = df.fillna(method="ffill").fillna(method="bfill")
    return df
sup_train = sup_train.groupby('Asset_ID').apply(reindex).reset_index(0, drop=True).sort_index()
sup_train = sup_train.iloc[(-14*pre_minute_beta):,:]
#add weight
sup_train['Weight'] = sup_train['Asset_ID'].map(add_weight_map)
sup_train.drop('Target',axis=1, inplace=True)

sup_train.set_index('Asset_ID',append=True, inplace=True)
##reorder by mod_order
sup_train = sup_train.reindex(list(zip(sup_train.index.get_level_values('timestamp'),
                           mod_order*pre_minute_beta)))

#######################################add lr_15,mkt_lr_15,crypto_index, beta,lr_mkt_resid
lr_15 = sup_train.groupby('Asset_ID').apply( 
        lambda x: log_return(x[['Close']],15)
        )
sup_train['lr_15'] = lr_15['Close']

mkt_lr_15 = sup_train.groupby('timestamp').apply( 
    lambda x: x[["lr_15", "Close"]].multiply(x["Weight"], axis="index").sum()
    )
mkt_lr_15.columns = ['Mkt_lrt_15','Crypto_Index']
firsts = sup_train.index.get_level_values('timestamp')
sup_train[['Mkt_lrt_15','Crypto_Index']] = mkt_lr_15.loc[firsts].values

####make beta,lr_mkt_resid, placeholder
sup_train[f"beta_{psets['beta_s']}"] = 0
sup_train[f"beta_{psets['beta_l']}"] = 0

sup_train.iloc[-14:,:]

Count          Open          High           Low  \
timestamp  Asset_ID                                                     
1642982400 3          457.0      1.124904      1.126200      1.119907   
           2          139.0    302.506000    302.770000    301.800000   
           0          420.0    383.901333    384.003000    382.371000   
           1         2917.0  36262.038571  36302.000000  36176.450000   
           4          227.0      0.141784      0.141900      0.141300   
           5          468.0      2.272950      2.277700      2.260000   
           7          280.0     25.128433     25.191000     24.984000   
           6         2510.0   2540.678571   2543.100000   2533.960000   
           8           49.0      0.814567      0.815300      0.811200   
           9          214.0    112.293000    112.470000    111.840000   
           10          25.0   1833.565000   1834.700000   1831.050000   
           13         235.0      0.057520      0.057618      0.057389   
           12         492.0      0.200940      0.201066      0.199352   
           11          40.0    156.764000    156.930000    156.200000   

                            Close        Volume          VWAP    Weight  \
timestamp  Asset_ID                                                       
1642982400 3             1.121151  2.499232e+05      1.123218  0.107797   
           2           302.112000  2.096457e+02    302.422931  0.058657   
           0           382.930000  1.490571e+03    383.253768  0.105286   
           1         36221.987143  1.100532e+02  36247.575361  0.165850   
           4             0.141391  1.009688e+06      0.141649  0.086971   
           5             2.264800  2.021940e+05      2.269373  0.033911   
           7            25.028067  6.709816e+03     25.093705  0.050867   
           6          2536.151429  1.057766e+03   2538.323926  0.144188   
           8             0.812400  6.019270e+03      0.813656  0.026874   
           9           111.983333  1.243860e+03    112.127619  0.058657   
           10         1833.018333  4.190290e+00   1833.173692  0.026874   
           13            0.057478  8.988253e+05      0.057519  0.043830   
           12            0.200093  2.983796e+06      0.200413  0.050867   
           11          156.550000  7.439747e+01    156.638720  0.039370   

                        lr_15  Mkt_lrt_15  Crypto_Index  beta_6h  beta_2d  
timestamp  Asset_ID                                                        
1642982400 3        -0.001616   -0.001114   6494.648269        0        0  
           2         0.001066   -0.001114   6494.648269        0        0  
           0        -0.001288   -0.001114   6494.648269        0        0  
           1        -0.001337   -0.001114   6494.648269        0        0  
           4        -0.000066   -0.001114   6494.648269        0        0  
           5        -0.002785   -0.001114   6494.648269        0        0  
           7        -0.004492   -0.001114   6494.648269        0        0  
           6         0.000148   -0.001114   6494.648269        0        0  
           8        -0.001189   -0.001114   6494.648269        0        0  
           9        -0.001254   -0.001114   6494.648269        0        0  
           10       -0.002008   -0.001114   6494.648269        0        0  
           13       -0.000622   -0.001114   6494.648269        0        0  
           12       -0.002306   -0.001114   6494.648269        0        0  
           11       -0.000843   -0.001114   6494.648269        0        0

In [22]:
myapi = API(train_df)
from datetime import datetime 
start_time = datetime.now()

for df_test, df_pred in myapi:
    num_asset_test = df_test.shape[0]
    row_asset_id_map = dict(zip(df_test.row_id, df_test.Asset_ID))
    test_timestamp = df_test.timestamp.values[0]
    print(f"{test_timestamp}", end = "\r")
    
    timestamp_list = sup_train.index.get_level_values('timestamp').unique().values
    timestamp_list = np.append(timestamp_list,test_timestamp)
    #######################################format df_test 
    ###add weight and index
    df_test['Weight'] = df_test['Asset_ID'].map(add_weight_map)
    ###fillin missing assets as nan
    df_test.set_index(['timestamp','Asset_ID'],inplace=True)
    #df_test = df_test.reindex(list(zip([test_timestamp]*14,range(14))))
    df_test = df_test.reindex(list(zip([test_timestamp]*14,mod_order)))
    
    ########################################concat to sup_train, add lr_15,mkt_lr_15,crypto_index
    sup_train = pd.concat([sup_train,df_test.drop('row_id',axis=1)],join='outer')
    test_lr_15 = sup_train.loc[timestamp_list[[-16,-1]]].groupby('Asset_ID').apply(
        lambda x: np.log(x[['Close']]).diff()
    )#same as mod_order
    sup_train.loc[test_timestamp, 'lr_15'] = test_lr_15.loc[test_timestamp,'Close'].values
    sup_train.loc[test_timestamp, ['Mkt_lrt_15','Crypto_Index']] = sup_train.loc[test_timestamp, ["lr_15", "Close"]].multiply(sup_train.loc[test_timestamp,"Weight"], axis="index").sum(skipna=True).values
    
    ########################################beta_sl, reorder by mod_order
    beta_s = sup_train[['lr_15','Mkt_lrt_15']].iloc[-14*(beta_sw):,:].groupby('Asset_ID').apply(
        lambda x: beta_resid(x,beta_sw)
    ).rename(f"beta_{psets['beta_s']}").reindex(mod_order)
    beta_l = sup_train[['lr_15','Mkt_lrt_15']].iloc[-14*(beta_lw):,:].groupby('Asset_ID').apply(
        lambda x: beta_resid(x,beta_lw)
    ).rename(f"beta_{psets['beta_l']}").reindex(mod_order)
    sup_train.loc[test_timestamp, [f"beta_{psets['beta_s']}",f"beta_{psets['beta_l']}"]] = \
        pd.concat([beta_s, beta_l],axis=1).values
    #########################################fill in missing assets as forward
    if num_asset_test <14:
        #ffill in missing
        sup_train = sup_train.groupby('Asset_ID').apply(lambda x: x.fillna(method="ffill")).iloc[14:,:]
    else:
        sup_train = sup_train.iloc[14:,:]
    #######################################add features to test timestamp, make sure no nan in this lstm window
    sup_train2 = sup_train.iloc[(-14*(pre_minute+lstm_window)):,:].copy()
    xx_test=sup_train2.groupby('Asset_ID').apply(
        lambda x: get_features(x,psets)
    ).iloc[(-14*lstm_window):,:]
    ########################################reorder and make np.array format
    #xx_test = xx_test.reindex(list(zip(xx_test.index.get_level_values('timestamp'),mod_order*lstm_window)))
    xx_test = np.array(xx_test[features_list])#(210,27)
    xx_test = xx_test.reshape(-1, 14, 27) #(15, 14, 27), len(features_list)
    xx_test = np.expand_dims(xx_test, axis=0) #(none , 15, 14, 27)
    #######################################make prediction
    y_pred = model.predict(xx_test).squeeze()#.reshape(-1, 1).squeeze()
    y = dict(zip(mod_order, y_pred))#dict(asset_id:pred)
    #match with row_id
    df_pred['Target']= [y[row_asset_id_map[r]] for r in df_pred['row_id']]
    myapi.predict(df_pred)

time_elapsed = datetime.now() - start_time
print('Time elapsed total (hh:mm:ss.ms) {}'.format(time_elapsed))
print(f'time elapsed per iteration {time_elapsed/myapi.init_num_times}')
print(f'Submission time estimate {129600*time_elapsed/myapi.init_num_times}')

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set. ;)


2022-01-29 13:07:53.082799: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-29 13:07:53.544142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-29 13:07:53.553371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-29 13:07:53.570444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-29 13:07:53.581244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-29 13:07:53.628054: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-01-29 13:07:53.638594: I tensorflow/core/grappler/optimizers/cust

Time elapsed total (hh:mm:ss.ms) 0:00:08.357040
time elapsed per iteration 0:00:00.139284
Submission time estimate 5:00:51.206400


In [23]:
model.predict(xx_test).squeeze()#.reshape(-1, 1).squeeze()

array([ -15.566902,  -57.58383 ,  139.34952 ,   47.41201 , -109.88918 ,
        -52.47073 ,  -94.27613 ,   64.74198 ,    4.811249,  -27.165201,
       -164.55385 ,   79.96053 , -104.35292 ,   76.297874], dtype=float32)

In [24]:
##scoring
id_2_weight = dict(zip(df_asset_details.Asset_ID, df_asset_details.Weight))
df, score = myapi.score(id_2_weight)
print(f"Your LB score is {round(score, 4)}")
myapi.predictions

Your LB score is 0.1345


[            row_id      Target
 22221694  22221694  -13.346309
 22221695  22221695  -54.405754
 22221696  22221696  132.684601
 22221697  22221697   46.569695
 22221698  22221698  -98.519554
 22221699  22221699  -50.704639
 22221700  22221700  -83.139702
 22221701  22221701   60.491035
 22221702  22221702   10.514942
 22221703  22221703  -24.776482
 22221704  22221704 -136.188538
 22221705  22221705   73.265579
 22221706  22221706  -93.359604
 22221707  22221707   71.082108,
             row_id      Target
 22221708  22221708  -13.536757
 22221709  22221709  -54.801071
 22221710  22221710  133.810730
 22221711  22221711   46.813873
 22221712  22221712  -99.824730
 22221713  22221713  -50.966511
 22221714  22221714  -84.265060
 22221715  22221715   61.007141
 22221716  22221716   10.225784
 22221717  22221717  -24.997080
 22221718  22221718 -139.397400
 22221719  22221719   74.011436
 22221720  22221720  -94.422806
 22221721  22221721   71.733582,
             row_id      Target
 22221